# MDETR - Modulated Detection for End-to-End Multi-Modal Understanding

> Made for running on Google Colab



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install timm==0.4.5 transformers==4.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 29.3 MB/s 
     |████████████████████████████████| 2.1 MB 51.7 MB/s 
     |████████████████████████████████| 880 kB 62.0 MB/s 
     |████████████████████████████████| 3.3 MB 61.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=b75a44384b26f5f224456166ca40aa921f4f3b28dd093899129e98d27e51fad5
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [3]:
import re
import torch
import pickle

import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as T

from collections import defaultdict
from skimage.measure import find_contours
from PIL import Image
from matplotlib.patches import Polygon
from tqdm import tqdm

torch.set_grad_enabled(False);

In [7]:
!cp /content/drive/MyDrive/BCUFR/dataset_for_phrase_grounding/dataset_for_phrase_grounding.zip dataset.zip &&  unzip -q dataset.zip 
dataset = pickle.load(open('dataset.p','rb'))


In [4]:
# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [5]:
# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image

def plot_results(pil_img, scores, boxes, labels, masks=None):
    plt.figure(figsize=(16,10))
    np_image = np.array(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    if masks is None:
      masks = [None for _ in range(len(scores))]
    # assert len(scores) == len(boxes) == len(labels) == len(masks)
    for s, (xmin, ymin, xmax, ymax), l, mask, c in zip(scores, boxes.tolist(), labels, masks, colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        text = f'{l}: {s:0.2f}'
        ax.text(int(xmin), int(ymin), text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))

        if mask is None:
          continue
        np_image = apply_mask(np_image, mask, c)

        padded_mask = np.zeros((mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
          # Subtract the padding and flip (y, x) to (x, y)
          verts = np.fliplr(verts) - 1
          p = Polygon(verts, facecolor="none", edgecolor=c)
          ax.add_patch(p)


    plt.imshow(np_image)
    plt.axis('off')
    plt.show()
    return np_image


def add_res(results, ax, color='green'):
    #for tt in results.values():
    if True:
        bboxes = results['boxes']
        labels = results['labels']
        scores = results['scores']
        #keep = scores >= 0.0
        #bboxes = bboxes[keep].tolist()
        #labels = labels[keep].tolist()
        #scores = scores[keep].tolist()
    #print(torchvision.ops.box_iou(tt['boxes'].cpu().detach(), torch.as_tensor([[xmin, ymin, xmax, ymax]])))
    
    colors = ['purple', 'yellow', 'red', 'green', 'orange', 'pink']
    
    for i, (b, ll, ss) in enumerate(zip(bboxes, labels, scores)):
        ax.add_patch(plt.Rectangle((b[0], b[1]), b[2] - b[0], b[3] - b[1], fill=False, color=colors[i], linewidth=3))
        cls_name = ll if isinstance(ll,str) else CLASSES[ll]
        text = f'{cls_name}: {ss:.2f}'
        ax.text(b[0], b[1], text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8), transform=ax.transAxes)

In [10]:
model, postprocessor = torch.hub.load('ashkamath/mdetr:main', 'mdetr_efficientnetB5', pretrained=True, return_postprocessor=True)
model = model.cuda()
model.eval();

Using cache found in /root/.cache/torch/hub/ashkamath_mdetr_main
Downloading: "https://zenodo.org/record/4721981/files/pretrained_EB5_checkpoint.pth" to /root/.cache/torch/hub/checkpoints/pretrained_EB5_checkpoint.pth


  0%|          | 0.00/2.53G [00:00<?, ?B/s]

Next, we retrieve an image on which we wish to test the model. Here, we use an image from the validation set of COCO

In [11]:
def plot_inference(im, caption):
  # mean-std normalize the input image (batch-size: 1)
  img = transform(im).unsqueeze(0).cuda()

  # propagate through the model
  memory_cache = model(img, [caption], encode_and_save=True)
  outputs = model(img, [caption], encode_and_save=False, memory_cache=memory_cache)

  # keep only predictions with 0.7+ confidence
  probas = 1 - outputs['pred_logits'].softmax(-1)[0, :, -1].cpu()
  keep = (probas > 0.7).cpu()

  # convert boxes from [0; 1] to image scales
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'].cpu()[0, keep], im.size)

  # Extract the text spans predicted by each box
  positive_tokens = (outputs["pred_logits"].cpu()[0, keep].softmax(-1) > 0.1).nonzero().tolist()
  predicted_spans = defaultdict(str)

  caption_word = re.findall('\w+|[^\sa-zA-Z0-9]+',caption)
  span = dict()
  for tok in positive_tokens:
    item, pos = tok
    if pos < 255:
      if item in span.keys():
        span[item].append(memory_cache["tokenized"].token_to_word(0, pos))
      else:
        span[item] = [memory_cache["tokenized"].token_to_word(0, pos)]
  for elem in span:
    for span_id in list(set(span[elem])):
      if span_id is not None:
        predicted_spans [elem] += " " + caption_word[min(span_id, len(caption_word) - 1)]


  labels = [predicted_spans [k] for k in sorted(list(predicted_spans .keys()))]
  # plot_results(im, probas[keep], bboxes_scaled, labels)
  return (probas[keep], bboxes_scaled, labels)

In [12]:
detection_results = []
for elem in tqdm(dataset):
  sentence = elem['title']['preprocessed']
  im = Image.open('img/' + elem["filename"]).convert('RGB')
  detection_results.append(plot_inference(im, sentence))



  0%|          | 0/1500 [00:00<?, ?it/s]/root/.cache/torch/hub/ashkamath_mdetr_main/models/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)
100%|██████████| 1500/1500 [03:47<00:00,  6.59it/s]


In [13]:
pickle.dump(detection_results, open('/content/drive/MyDrive/BCUFR/MDETR_title.p','wb'))